In [30]:
from google.cloud import bigtable
import pandas as pd
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/louischang/mykeys/braided-turbine-412903-60548abfc820.json"

def insert_sensor_data_to_bigtable(project_id, instance_id, table_id, row_key, column_family_id, sensor_data):
    client = bigtable.Client(project=project_id, admin=True)
    instance = client.instance(instance_id)
    table = instance.table(table_id)

    row = table.direct_row(row_key)
    for column_qualifier, value in sensor_data.items():
        if pd.notna(value):  # 檢查值是否非空
            # Bigtable 要求數據以字節形式
            value_bytes = str(value).encode('utf-8')  # 直接轉換為字節
            row.set_cell(column_family_id,
                         column_qualifier.encode('utf-8'),  # 確保列修飾符也是字節型態
                         value_bytes)
    try:
        row.commit()
        print(f"Data inserted for row key: {row_key}")
    except Exception as e:
        print(f"Failed to insert data for row key: {row_key}. Error: {e}")

# Connect to Bigtable and prepare the environment
project_id = "braided-turbine-412903"
instance_id = "cloudinfrahw4"
table_id = "sensor_data2"
column_family_id = 'sensor_details'
client = bigtable.Client(project=project_id, admin=True)
instance = client.instance(instance_id)
table = instance.table(table_id)

for index, row in df.iterrows():
    row_key = row['Sensor_ID']  # Assuming 'Sensor_ID' is used as the row key
    sensor_data = {
        "Model_Used": row['Model_Used'],
        "Notes": row['Notes'],
        "Humidity_Percentage": row['Humidity_Percentage'],
        "Temperature_in_Fahrenheit": row['Temperature_in_Fahrenheit'],
        "Wind_Speed_in_mph": row['Wind_Speed_in_mph'],
        "Wind_Direction": row['Wind_Direction']
    }
    # Convert row_key to bytes for Bigtable
    row_key_bytes = row_key.encode('utf-8')
    insert_sensor_data_to_bigtable(project_id, instance_id, table_id, row_key_bytes, column_family_id, sensor_data)


Data inserted for row key: b'AAAAAA'
Data inserted for row key: b'AAAAAB'
Data inserted for row key: b'AAAAAC'
Data inserted for row key: b'AAAAAD'
Data inserted for row key: b'AAAAAE'
Data inserted for row key: b'AAAAAF'
Data inserted for row key: b'AAAAAG'
Data inserted for row key: b'AAAAAH'
Data inserted for row key: b'AAAAAI'
Data inserted for row key: b'AAAAAJ'
Data inserted for row key: b'AAAAAK'
Data inserted for row key: b'AAAAAL'
Data inserted for row key: b'AAAAAM'
Data inserted for row key: b'AAAAAN'


In [31]:
from google.cloud import bigtable

def query_bigtable(project_id, instance_id, table_id):
    client = bigtable.Client(project=project_id, admin=True)
    instance = client.instance(instance_id)
    table = instance.table(table_id)
    
    partial_rows = table.read_rows()
    partial_rows.consume_all()
    
    print("Scanning for all sensors: ")
    for row_key, row in partial_rows.rows.items():
        print(f"Reading data for {row_key.decode('utf-8')}: ")
        for column_family_id, columns in row.cells.items():
            for column_qualifier, cell_list in columns.items():
                for cell in cell_list:
                    cell_value = cell.value.decode('utf-8')
                    cell_timestamp = cell.timestamp
                    print(f"\t\t{column_qualifier.decode('utf-8')}: {cell_value} ")

project_id = "braided-turbine-412903"
instance_id = "cloudinfrahw4"
table_id = "sensor_data2"
query_bigtable(project_id, instance_id, table_id)


Scanning for all sensors: 
Reading data for AAAAAA: 
		Humidity_Percentage: 0.5
		Model_Used: modnist-sensor
		Notes: newer type of sensor 
		Temperature_in_Fahrenheit: 45.0
		Wind_Direction: NW
		Wind_Speed_in_mph: 4 mph
Reading data for AAAAAB: 
		Humidity_Percentage: 0.4
		Model_Used: mednist-sensor
		Notes: our mid-age quality sensor
		Wind_Direction: SW
		Wind_Speed_in_mph: 5 mph
Reading data for AAAAAC: 
		Humidity_Percentage: 0.8
		Model_Used: oldnist-sensor
		Notes: our oldest sensor
Reading data for AAAAAD: 
		Humidity_Percentage: 0.9
		Model_Used: modnist-sensor
		Notes: newer type of sensor 
		Temperature_in_Fahrenheit: 95.0
		Wind_Direction: NE
		Wind_Speed_in_mph: 2 mph
Reading data for AAAAAE: 
		Humidity_Percentage: 0.4
		Model_Used: mednist-sensor
		Notes: our mid-age quality sensor
		Wind_Direction: SW
		Wind_Speed_in_mph: 7 mph
Reading data for AAAAAF: 
		Humidity_Percentage: 0.7
		Model_Used: mednist-sensor
		Notes: our mid-age quality sensor
		Wind_Direction: SW
		W